In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import pandas as pd


#driver = webdriver.Chrome(ChromeDriverManager().install())



def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "chromedriver"}
    return Browser("chrome", **executable_path, headless=False)
scrape_solutions ={}

def scrape(listings):
    browser = init_browser()
    

    url = "https://mars.nasa.gov/news/"
    browser.visit(url)

    html = browser.html
    soup = BeautifulSoup(html, "html.parser")

    listings["headline"] = soup.find("div", class_="content_title").get_text()
    listings["paragraph"] = soup.find("div", class_="article_teaser_body").get_text()

    return listings

In [2]:
scrape(scrape_solutions)

{'headline': "NASA's Briefcase-Size MarCO Satellite Picks Up Honors",
 'paragraph': 'The twin spacecraft, the first of their kind to fly into deep space, earn a Laureate from Aviation Week & Space Technology.'}

In [3]:
def scrape_marsimages(listings):
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser = init_browser()
    browser.visit(url)

    html = browser.html
    soup = BeautifulSoup(html, "html.parser")

    mars_images = soup.find("article", class_="carousel_item")
    listings["images"] = re.findall(r"'(.*?)'",str(mars_images), re.DOTALL)

    #listings["paragraph"] = soup.find("div", class_="article_teaser_body").get_text()

    return listings

In [4]:
scrape_marsimages(scrape_solutions)

{'headline': "NASA's Briefcase-Size MarCO Satellite Picks Up Honors",
 'paragraph': 'The twin spacecraft, the first of their kind to fly into deep space, earn a Laureate from Aviation Week & Space Technology.',
 'images': ['/spaceimages/images/wallpaper/PIA16682-1920x1200.jpg']}

In [5]:
print(scrape_solutions)

{'headline': "NASA's Briefcase-Size MarCO Satellite Picks Up Honors", 'paragraph': 'The twin spacecraft, the first of their kind to fly into deep space, earn a Laureate from Aviation Week & Space Technology.', 'images': ['/spaceimages/images/wallpaper/PIA16682-1920x1200.jpg']}


In [6]:
def mars_twitter(listings):
    url ="https://twitter.com/marswxreport?lang=en"
    browser = init_browser()
    browser.visit(url)

    html = browser.html
    soup = BeautifulSoup(html, "html.parser")

    twitter_tweets = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").get_text()
    listings ["tweets"]= twitter_tweets
   #listings["images"] = re.findall(r"'(.*?)'",str(mars_images), re.DOTALL)

    #listings["paragraph"] = soup.find("div", class_="article_teaser_body").get_text()

    return listings





In [7]:
mars_twitter(scrape_solutions)

{'headline': "NASA's Briefcase-Size MarCO Satellite Picks Up Honors",
 'paragraph': 'The twin spacecraft, the first of their kind to fly into deep space, earn a Laureate from Aviation Week & Space Technology.',
 'images': ['/spaceimages/images/wallpaper/PIA16682-1920x1200.jpg'],
 'tweets': 'InSight sol 365 (2019-12-06) low -98.8ºC (-145.8ºF) high -21.5ºC (-6.8ºF)\nwinds from the SSW at 5.7 m/s (12.7 mph) gusting to 20.2 m/s (45.2 mph)\npressure at 6.60 hPapic.twitter.com/JF2KNAksNr'}

In [8]:
#class="tablepress tablepress-id-p-mars"
def get_mars_table():
    url ="https://space-facts.com/mars/"
    browser = init_browser()
    browser.visit(url)

    html = browser.html
    soup = BeautifulSoup(html, "html.parser")

    mars_table = soup.find("table", class_="tablepress tablepress-id-p-mars")
    mars_table = str(mars_table)
    dfs = pd.read_html(mars_table)
    return dfs
    #listings ["tweets"]= twitter_tweets

In [9]:
scrape_solutions["table"]=pd.DataFrame(get_mars_table()).to_html()
scrape_solutions["table"]

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>0                              1\n0  Equatorial Diameter:                       6,792 km\n1       Polar Diameter:                       6,752 km\n2                 Mass:  6.39 × 10^23 kg (0.11 Earths)\n3                Moons:            2 (Phobos &amp; Deimos)\n4       Orbit Distance:       227,943,824 km (1.38 AU)\n5         Orbit Period:           687 days (1.9 years)\n6  Surface Temperature:                   -87 to -5 °C\n7         First Record:              2nd millennium BC\n8          Recorded By:           Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [10]:
def get_mars_picture(url):
    
    browser = init_browser()
    browser.visit(url)

    html = browser.html
    soup = BeautifulSoup(html, "html.parser")

    picture = soup.find("div", class_="downloads")
    title = soup.find("h2", class_= "title").get_text()
    
    return (title, picture.a["href"])


In [11]:
scrape_solutions["picture1"] = get_mars_picture("https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced")
scrape_solutions["picture2"] = get_mars_picture("https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced")
scrape_solutions["picture3"] = get_mars_picture("https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced")
scrape_solutions["picture4"] = get_mars_picture("https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced")

In [ ]:
from flask import Flask, render_template, redirect



app = Flask(__name__)

# Use flask_pymongo to set up mongo connection
#app.config["MONGO_URI"] = "mongodb://localhost:27017/craigslist_app"


# Or set inline
# mongo = PyMongo(app, uri="mongodb://localhost:27017/craigslist_app")


@app.route("/")
def index():
  #  listings = mongo.db.listings.find_one()
    
    return render_template("index.html", scrape_solutions=scrape_solutions)


@app.route("/scrape")
def scraper():
    scrape_solutions["picture1"] = get_mars_picture("https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced")
    scrape_solutions["picture2"] = get_mars_picture("https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced")
    scrape_solutions["picture3"] = get_mars_picture("https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced")
    scrape_solutions["picture4"] = get_mars_picture("https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced")
    scrape_solutions["table"]=pd.DataFrame(get_mars_table()).to_html()
    mars_twitter(scrape_solutions)
    scrape_marsimages(scrape_solutions)
    scrape(scrape_solutions)
    return redirect("/", code=302)


#if __name__ == "__main__":
app.run(debug=True,use_reloader=False,port=3000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:3000/ (Press CTRL+C to quit)


In [ ]:
%tb